In [ ]:
from google.colab import drive
drive.mount

In [ ]:
!pip install easyocr

In [ ]:
import os
import csv
import cv2
import imutils
import easyocr

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
#SHOWING THE STEPS FOR IMAGE PROCESSING

#taking in our image input and resizing its width to 300 pixels
image = cv2.imread("/content/GA7.jpg")         
image = imutils.resize(image, width=300 )
print("original image")
cv2_imshow(image)
#cv2.waitKey(0)

#converting the input to greyscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
print("greyed image")
cv2_imshow( gray_image)
# cv2.waitKey(0)

#Reducing the noise in the greyscale image
gray_image = cv2.bilateralFilter(gray_image, 11, 17, 17) 
print("smoothened image")
cv2_imshow(gray_image)
# cv2.waitKey(0)

#Detecting the edges of the smoothened image
edged = cv2.Canny(gray_image, 30, 200) 
print("edged image")
cv2_imshow(edged)
# cv2.waitKey(0)

#Finding the contours from the edged image
cnts,new = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
image1=image.copy()
cv2.drawContours(image1,cnts,-1,(0,255,0),3)
print("contours")
cv2_imshow(image1)
# cv2.waitKey(0)

#Sorting the identified contours
cnts = sorted(cnts, key = cv2.contourArea, reverse = True) [:30]
screenCnt = None
image2 = image.copy()
cv2.drawContours(image2,cnts,-1,(0,255,0),3)
print("Top 30 contours")
cv2_imshow(image2)
# cv2.waitKey(0)

#Finding the contour with four sides
i=7
for c in cnts:
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * perimeter, True)
        if len(approx) == 4: 
                screenCnt = approx

#Cropping the rectangular part identified as license plate
                x,y,w,h = cv2.boundingRect(c) 
                new_img=image[y:y+h,x:x+w]
                cv2.imwrite('./'+str(i)+'.png',new_img)
                i+=1
                break

#Drawing the selected contour on the original image
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 3)
print("image with detected license plate")
cv2_imshow(image)
#cv2.waitKey(0)
Cropped_loc=cv2.imread('/content/7.png')
cv2_imshow(Cropped_loc)
# using ocr to read text
reader = easyocr.Reader(['en'])
result = reader.readtext(Cropped_loc)
print("Detected license plate is")
print(result[0][1])


In [ ]:
# IMPLEMENTING CODE ON DATASET
root="/content/drive/MyDrive/test"
f=open("Pred.csv",'a')
writer=csv.writer(f)
writer.writerow(["Image","Predicted License plate number"])
for folder in os.listdir(root):
  if(folder !='LICENSE.txt'):
    for file in os.listdir(root+'/'+folder):
      path=root+'/'+folder+'/'+file

      #taking in our image input and resizing its width to 300 pixels
      try:
        image = cv2.imread(path)           
        image = imutils.resize(image, width=300 )
        cv2_imshow(image)
        #cv2.waitKey(0)
      except:
        continue

      #converting the input to greyscale
      gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      # cv2.imshow("greyed image", gray_image)
      # cv2.waitKey(0)

      #Reducing the noise in the greyscale image
      gray_image = cv2.bilateralFilter(gray_image, 11, 17, 17) 
      # cv2.imshow("smoothened image", gray_image)
      # cv2.waitKey(0)

      #Detecting the edges of the smoothened image
      edged = cv2.Canny(gray_image, 30, 200) 
      # cv2.imshow("edged image", edged)
      # cv2.waitKey(0)

      #Finding the contours from the edged image
      cnts,new = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
      image1=image.copy()
      cv2.drawContours(image1,cnts,-1,(0,255,0),3)
      # cv2.imshow("contours",image1)
      # cv2.waitKey(0)

      #Sorting the identified contours
      cnts = sorted(cnts, key = cv2.contourArea, reverse = True) [:30]
      screenCnt = None
      image2 = image.copy()
      cv2.drawContours(image2,cnts,-1,(0,255,0),3)
      # cv2.imshow("Top 30 contours",image2)
      # cv2.waitKey(0)

      #Finding the contour with four sides
      i=7
      for c in cnts:
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * perimeter, True)
        if len(approx) == 4: 
                screenCnt = approx

                #Cropping the rectangular part identified as license plate
                x,y,w,h = cv2.boundingRect(c) 
                new_img=image[y:y+h,x:x+w]
                cv2.imwrite('./'+str(i)+'.png',new_img)
                i+=1
                break

      #Drawing the selected contour on the original image
      cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 3)
      cv2_imshow(image)
      #cv2.waitKey(0)

      #cropped license plate
      Cropped_loc = './7.png'
      cv2_imshow(cv2.imread(Cropped_loc))
      #cv2.waitKey(0)

      # using ocr to read text
      reader = easyocr.Reader(['en'])
      result = reader.readtext(Cropped_loc)
      try:
        plate=result[0][1]
        print(plate)
        writer.writerow([file,plate])
      except IndexError:
        print(result)
        writer.writerow([file,"null"])

    f.close()

In [ ]:
import pandas as pd
df=pd.read_csv("/content/Pred.csv")
df['Predicted License plate number'].isna().value_counts()